# EDA

In [ ]:
import pandas as pd
import json

In [ ]:
# 멀티턴 eval_id
multiturn_ids = [107, 42, 43, 97, 243, 66, 98, 295, 290, 68, 86, 89, 306, 39, 33, 249, 54, 3, 44, 278]
# 일상대화 eval_id
chitchat_list = [276, 261, 283, 32, 94, 90, 220, 245, 229, 247, 67, 57, 2, 227, 301, 222, 83, 64, 103, 218]

## eval

#### eval load

In [ ]:
import pandas as pd

def load_json(file_path):
    # jsonl 파일을 DataFrame으로 읽어오기
    # lines=True 옵션이 핵심입니다.
    try:
        df = pd.read_json(file_path, lines=True)
        print("성공적으로 DataFrame을 로드했습니다.")
        print(df.head())
        return df
    except FileNotFoundError:
        print(f"오류: 파일을 찾을 수 없습니다. 경로를 확인하세요: {file_path}")
    except Exception as e:
        print(f"데이터 로드 중 오류 발생: {e}")

In [ ]:
file_path = '../data/eval.jsonl'
eval_df = load_json(file_path)
eval_df

#### 멀티턴 메시지 확인

In [ ]:
eval_df['count'] = eval_df['msg'].apply(lambda x: len(x))
eval_df[eval_df['count']>1]

## Documents

#### documents load

In [ ]:
file_path = '../data/documents.jsonl'
doc_df = load_json(file_path)

#### 카테고리 확인

In [ ]:
pattern_to_remove = '_train|_valid|_test'
doc_df['src_new'] = doc_df['src']
doc_df['src_new'] = doc_df['src_new'].str.replace(pattern_to_remove, '', regex=True)
doc_df

In [ ]:
num_categories = doc_df['src_new'].value_counts()
num_categories

## Submission

In [ ]:
def get_submission_df(file_path):
    data_list = [] # 파싱된 딕셔너리들을 저장할 리스트
    print(f"'{file_path}' 파일을 JSON Lines 형식으로 파싱합니다...")

    # 파일을 '읽기(r)' 모드로 열기
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f):
                stripped_line = line.strip()

                if not stripped_line:
                    continue
                
                # 각 줄을 JSON 문자열로 간주하여 파이썬 딕셔너리로 변환
                try:
                    record = json.loads(stripped_line)
                    data_list.append(record)
                except json.JSONDecodeError as e:
                    print(f"⚠️ {file_path} 파일의 {line_num + 1}번째 줄 파싱 오류: {e}")
                    print(f"    문제의 줄 내용: {stripped_line[:50]}...")
                    continue

        # 파싱된 리스트를 Pandas DataFrame으로 변환
        df_parsed = pd.DataFrame(data_list)
        
        print("\n🎉 파일 로드 및 파싱 성공!")
        print(df_parsed.info())
        return df_parsed
    except FileNotFoundError:
        print(f"❌ 오류: 지정된 경로에 파일 '{file_path}'이(가) 없습니다. 경로를 확인해주세요.")

In [ ]:
df = get_submission_df("sample_submission.csv")
df

In [ ]:
empty_list_df = df[df["references"].str.len() == 0]
referenced_list_df = df[df["references"].str.len() > 0]

In [ ]:
print(len(empty_list_df))
print(len(referenced_list_df))

In [ ]:
# 멀티턴
multi_df = df[df['eval_id'].isin(multiturn_ids)]
multi_df

In [ ]:
tmp_df = pd.merge(df, eval_df, on='eval_id', how='left')
result_df = tmp_df[['eval_id', 'standalone_query', 'msg']]
result_df